<a href="https://colab.research.google.com/github/MatheusMRech/ETL-Dataiku-DSS/blob/master/REBLEED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Professor Jonathan pensei q esta apresentação em tópicos poderia ajudar na apresentação.

___
Dataset usados

Original: 

Prospectivo:
___
In the Methods section, you can describe the Python code as follows:

In the Methods section, you can update the description of the Python code to include normalization of the numeric values and one-hot encoding of categorical values:

1. Data Loading and Preprocessing: Load the dataset containing medical records from a suitable file format (e.g., CSV, Excel) using an appropriate library such as pandas. Drop the rows with missing outcomes (death and rebleeding) from the original and prospective datasets.

2. Missing Data Imputation: Perform K-NN imputation on the original and prospective datasets to handle missing data in the remaining features. Use an appropriate imputation library, such as the KNNImputer from scikit-learn, to fill in missing values based on the K-nearest neighbors in the feature space.

3. Data Preprocessing: Normalize the numeric values in the original and prospective datasets using an appropriate method, such as StandardScaler or MinMaxScaler, from the scikit-learn library. One-hot encode the categorical values using the OneHotEncoder class from scikit-learn.

4. Correlation Analysis: Compute Pearson correlation coefficients between each of the 36 variables and the two outcomes, death and rebleeding, to assess the strength and direction of their linear relationships.

5. Data Preparation: Split the dataset into features (X) and outcomes (y_death and y_rebleeding) for further processing. The features include all the variables except for the outcomes, death and rebleeding.

6. Repeated Stratified Cross-Validation: Implement Repeated Stratified Cross-Validation with 5 folds and 5 repeats for model evaluation. Stratified Cross-Validation ensures that the proportion of outcomes is maintained in each fold, and repeating the process helps to reduce the variability of the results. This can be done using the RepeatedStratifiedKFold class from the scikit-learn library.

7. ANN Training and Evaluation: Create two Artificial Neural Networks (ANNs) with one hidden layer of 10 neurons each. Train and evaluate both ANNs using the Repeated Stratified Cross-Validation approach. For each fold and repeat, train the ANNs for 500 epochs using the backpropagation algorithm. The first ANN is trained with the 'death' outcome, while the second ANN is trained with the 'rebleeding' outcome. This can be done using the Keras library or the MLPClassifier from scikit-learn.

8. Model Performance: Aggregate the performance metrics such as accuracy, sensitivity, specificity, etc., across all folds and repeats to assess the overall performance of the models. These metrics can be calculated using functions from the scikit-learn library.

9. ROC Curve Analysis: Plot Receiver Operating Characteristic (ROC) curves for both ANNs based on the aggregated results from the Repeated Stratified Cross-Validation to visually represent the trade-off between sensitivity and specificity. Calculate the Area Under the Curve (AUC) as a measure of the models' performance. This can be done using the roc_curve and roc_auc_score functions from the scikit-learn library.

10. Applying Trained Models to Prospective Dataset: After training the ANNs using Repeated Stratified Cross-Validation on the original dataset, apply the same preprocessing methods (normalization and one-hot encoding) that were applied on the original dataset to the prospective dataset. Then, use the trained models to predict the death and rebleeding outcomes on the prospective dataset. This can be done by selecting the models with the best average performance during cross-validation and using the 'predict' function available in the scikit-learn library or Keras library.

11. Evaluating Model Performance on Prospective Dataset: Evaluate the performance of the models on the prospective dataset by calculating performance metrics such as confusion matrix, accuracy, sensitivity, specificity, etc. This will give an indication of how well the trained models generalize to new, unseen data. The performance metrics can be compared to the performance metrics obtained during training and evaluation on the original dataset to determine if there is any significant difference in performance.

12. Model Interpretability: After training and evaluating the ANNs, analyze the trained models' interpretability to gain insights into the relationships between the input features and the outcomes. This can be done using techniques such as feature importance ranking or partial dependence plots.


Python

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# Load original dataset
df_original = pd.read_csv('original_dataset.csv')

# Drop rows with missing outcomes
df_original = df_original.dropna(subset=['death', 'rebleeding'])

# Split dataset into features and outcomes
X_original = df_original.drop(['death', 'rebleeding'], axis=1)
y_death_original = df_original['death']
y_rebleeding_original = df_original['rebleeding']

# Impute missing values using K-NN
imputer = KNNImputer(n_neighbors=5)
X_original = pd.DataFrame(imputer.fit_transform(X_original), columns=X_original.columns)

# Normalize numeric values
scaler = StandardScaler()
X_original[X_original.select_dtypes(include=['float64']).columns] = scaler.fit_transform(X_original.select_dtypes(include=['float64']))

# One-hot encode categorical values
encoder = OneHotEncoder(sparse=False)
X_original_encoded = pd.DataFrame(encoder.fit_transform(X_original.select_dtypes(include=['object'])), columns=encoder.get_feature_names(X_original.select_dtypes(include=['object']).columns))

# Concatenate encoded categorical features with numeric features
X_original = pd.concat([X_original_encoded, X_original.select_dtypes(include=['float64', 'int64'])], axis=1)

# Repeated Stratified Cross-Validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# Create ANNs with one hidden layer of 10 neurons each
model_death = Sequential()
model_death.add(Dense(10, activation='relu', input_shape=(X_original.shape[1],)))
model_death.add(Dense(1, activation='sigmoid'))
model_death.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_rebleeding = Sequential()
model_rebleeding.add(Dense(10, activation='relu', input_shape=(X_original.shape[1],)))
model_rebleeding.add(Dense(1, activation='sigmoid'))
model_rebleeding.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train and evaluate ANNs using Repeated Stratified Cross-Validation
results_death = []
results_rebleeding = []
for train_idx, test_idx in cv.split(X_original, y_death_original):
    X_train, X_test = X_original.iloc[train_idx], X_original.iloc[test_idx]
    y_train_death, y_test_death = y_death_original.iloc[train_idx], y_death_original.iloc[test_idx]
    y_train_rebleeding, y_test_rebleeding = y_rebleeding_original.iloc[train_idx], y_rebleeding_original.iloc[test_idx]

    # Fit model on death outcome
    model_death.fit(X_train, y_train_death, epochs=500, verbose=0)
    y_pred_death = model_death.predict_classes(X_test)
    results_death.append(accuracy_score(y_test_death, y_pred_death))

    # Fit model on rebleeding outcome
    model_rebleeding.fit(X_train, y_train_rebleeding, epochs=500, verbose=0)
    y_pred_rebleeding = model_rebleeding.predict_classes(X_test)
    results_rebleeding.append(accuracy_score(y_test


MATLAB

In [ ]:
#**Load the data and split into training and testing sets**
```matlab
% Load the data (assuming the data is in a CSV format)
data = readtable('your_data_file.csv');

% Split the dataset into training (80%) and testing (20%) sets
rng('default'); % For reproducibility
cv = cvpartition(height(data), 'HoldOut', 0.2);
idx = cv.test;

% Separate input features (X) and target variable (y)
X = data(:, 1:end-1);
y = data.rebleed;

% Training and testing sets
X_train = X(~idx,:);
X_test = X(idx,:);
y_train = y(~idx,:);
y_test = y(idx,:);
```

#**Preprocess the data**
```matlab
% Preprocess numeric and categorical features
numericVars = varfun(@isnumeric, X_train, 'output', 'uniform');
categoricalVars = ~numericVars;

#Standardize numeric features
z = zscore(X_train{:, numericVars});
X_train{:, numericVars} = z;

#Convert categorical features to dummy variables
X_train = dummyvar(X_train);

#Apply the same transformations to the test set
z_test = zscore(X_test{:, numericVars});
X_test{:, numericVars} = z_test;
X_test = dummyvar(X_test);
```

#**Handle class imbalance with SMOTE (using the SMOTE package)**
```matlab
% Install the SMOTE package from https://github.com/zygmuntz/smote
addpath('/path/to/smote/folder');

% Perform SMOTE
[X_train_smote, y_train_smote] = smote(X_train, y_train);
```

#**Train the models**
```matlab
% Define the models: XGBoost, LightGBM, and MLP
t = templateTree('Surrogate','on');
xgboost = fitcensemble(X_train_smote, y_train_smote, 'Method', 'Bag', 'Learners', t);
lightgbm = fitcensemble(X_train_smote, y_train_smote, 'Method', 'LSBoost', 'Learners', t);
mlp = fitcnet(X_train_smote, y_train_smote);

#Train the models (use grid search and cross-validation to find the best hyperparameters)
#For XGBoost, LightGBM, and MLP, refer to the MATLAB documentation for hyperparameter tuning.
```

#**Model selection**
```matlab
% Compare the models using AUC-ROC scores on cross-validation
% Use kfoldLoss with 'LossFun', 'logloss' option for each model and calculate AUC-ROC
% Select the model with the highest AUC-ROC score as the best model
```

#**Model validation**
```matlab
% Validate the performance of the best model on the test set
% Calculate the AUC-ROC, accuracy, sensitivity, specificity, recall, and F1 score
% Use the perfcurve and confusionmat functions for these calculations
```

Análise Fernanda - Eu mudei um poucos os metodos em relacao aos que ela usou originalmente para tentar deixar o modelo mais robusto. A análise que ela descreveu originalmente é esta abaixo. Também forneço a versão em MATLAB e em python.

In [ ]:
In the Methods section, you can describe the MATLAB code as follows:

1. Data Loading: The dataset containing medical records is assumed to be stored in a '.mat' file. The data is loaded into MATLAB workspace.

2. Correlation Analysis: Pearson correlation coefficients are computed between each of the 36 variables and the two outcomes, death and rebleeding, to assess the strength and direction of their linear relationships.

3. Data Preparation: The dataset is split into features (X) and outcomes (y_death and y_rebleeding) for further processing. The features include all the variables except for the outcomes, death and rebleeding.

4. Data Splitting: The dataset is split into training (70%), testing (15%), and validation (15%) subsets using stratified sampling, ensuring the proportion of outcomes is maintained in each subset.

5. ANN Training: Two Artificial Neural Networks (ANNs) are created with one hidden layer of 10 neurons each. Both ANNs are trained for 500 epochs using the backpropagation algorithm. The first ANN is trained with the 'death' outcome, while the second ANN is trained with the 'rebleeding' outcome.

6. Model Evaluation: Both ANNs are evaluated on the test dataset by generating predictions for the death and rebleeding outcomes. Performance metrics, such as confusion matrix and classification report (accuracy, sensitivity, specificity, etc.), are computed to assess the performance of the models.

7. ROC Curve Analysis: Receiver Operating Characteristic (ROC) curves are plotted for both ANNs to visually represent the trade-off between sensitivity and specificity. The Area Under the Curve (AUC) is calculated as a measure of the models' performance.

Throughout the Methods section, it is essential to provide a clear and concise description of the steps involved in the analysis. This will enable readers to understand the process and reproduce the results if needed.